# 1. **Develop solution:**  Credit Fraud Detection for financial business

## 1.1. Introduction

In this part of the series, we will train an Machine learning or Deep learning based model (implemented in Keras) in for Anomaly Detection in credit card transaction data. The trained model will be evaluated on pre-labeled and anonymized dataset.

Ready? Let's go!

## 1.2.  Install the modeling requirements and libraries

First, we'll need to **install some libraries** that are not part of our container image. Normally, **Red Hat OpenShift Data Science** is already taking care of this for you, based on what it detects in the code. **Red Hat OpenShift Data Science** will reinstall all those libraries for you every time you launch the notebook!

In case you're using this notebook in a different environment, or just to make sure everything is ready, you can run the following cell to install OpenCV (a library to work with images) and Keras (an abstraction layer over Tensorflow).

In [2]:
%pip install -r requirements.txt 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 165.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2
Note: you may need to restart the kernel to use updated packages.


## 1.3.  Importing the needed libraries and packages
Of course, we'll need to import various packages. They are either built in the notebook image you are running, or have been installed in the previous step.

In [3]:
import os, sys; sys.path.append(os.path.dirname(os.getcwd()))
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time


from src.dataloading.read_dataset import readData
from src.features.data_preprocessing import preprocessData
from src.visualization.visualize import visualizeData
from src.modules.build_model import buildModel
from src.modules.train_model import  trainModel
from src.modules.predict_model import predictor
from src.hyper_parameters.hps import get_hyper_paras
from src.github_commands.git_utils import gitCommands



## 1.4. Initialize some hyper-parameters

In [4]:
dataPath,BATCH,EPOCHS,model_Type,model_Name, model_dir,refRepoName,sourceRepoName,refRepoDir,sourceRepoDir,scalerPicklePath = get_hyper_paras()
dataPath,BATCH,EPOCHS,model_Type,model_Name, model_dir,refRepoName,sourceRepoName,refRepoDir,sourceRepoDir,scalerPicklePath

('/opt/app-root/src/FSI-Fraud-Detection-Workshop/data/raw/creditcard.csv',
 64,
 100,
 'ml',
 'LogisticRegression',
 '/opt/app-root/src/FSI-Fraud-Detection-Inference/models/finalized_ml_model.sav',
 'FSI-Fraud-Detection-Inference',
 'FSI-Fraud-Detection-Workshop',
 '/opt/app-root/src/FSI-Fraud-Detection-Inference',
 '/opt/app-root/src/FSI-Fraud-Detection-Workshop',
 '/opt/app-root/src/FSI-Fraud-Detection-Inference/models/scaler.pkl')

## 1.5. Reading the  Dataset

The data set is available on Kaggle for download - https://www.kaggle.com/dalpozz/creditcardfraud

In [ ]:
data = readData(dataPath).readDataFrame()
data.head()
data.columns

In [ ]:
data.columns

## 1.6.  Data Preprocessing

We need to resize the data to make them ready for feeing to our model

In [ ]:
train_org, test, val, train = preprocessData(data,scalerPicklePath).dataPreProcessing()
train_org[1].shape, test[1].shape, val[1].shape, train[1].shape

## 1.7.  Design and compile the model 

In [ ]:
clf = buildModel(train_data=train, modelType = model_Type).setupModel()
# clf = buildModel(train_data=train, modelType = 'ml').setupModel()
# clf = buildModel(train_data=train,modelType = 'dl').setupModel()

## 1.8.  Training the Model

In [ ]:
clf = trainModel(clf,train_data =train,val_data=val,modelType=model_Type,epochs=100,savePath=model_dir).modelTraining()
# clf = trainModel(clf,train_data=train,savePath =model_dir, modelType = 'ml' ).modelTraining()
# clf = trainModel(clf,train_data =train,val_data=val,modelType='dl',epochs=100,savePath=model_dir).modelTraining()

## 1.9.  Test Model based on validation data

In [ ]:
predictor(clf = clf,data=test, modelType=model_Type).predict()
# predictor(clf = clf,data=test, modelType='ml').predict()
# predictor(clf = clf,data=test, modelType='dl').predict()

## 1.10. Update the Private\Public git repo

In [ ]:
temp=time.localtime(time.time())
uploaddate= 'Update with the latest change ' + str(temp[0])+'_'+str(temp[1])+'_'+str(temp[2])+'_'+str(temp[3])+'_'+str(temp[4])

gitCommands(repo_dir = sourceRepoDir,repo_name = sourceRepoName,git_email= os.environ['GIT_EMAIL'], git_username = os.environ['GIT_USER_NAME'].lower(), git_token = os.environ['GIT_TOKEN'], commit_message = uploaddate, file_name = '.').gitPush()

# 2. Deploy solution for Credit Fraud Detection 

## 2.1 Push the change to the inference repo 

In [ ]:
# gitCommands(repo_dir = repoDir,repo_name = repoName,git_email= email, git_username = username, git_token = token, commit_message = uploaddate, file_name = '.').gitPush()
gitCommands(repo_dir = refRepoDir,repo_name = refRepoName,git_email= os.environ['GIT_EMAIL'], git_username = os.environ['GIT_USER_NAME'].lower(), git_token = os.environ['GIT_TOKEN'], commit_message = uploaddate, file_name = '.').gitPush()

## 2.2 Deploy as an app with seldon

In [ ]:
import os, sys; sys.path.append(os.path.dirname(os.getcwd()))
import subprocess

current_path= os.path.dirname(os.getcwd())
inference_path = current_path.replace('Workshop','Inference')
# sys.path.append(inference_path+"/deploy/")
os.system('python ' +inference_path+"/deploy/" +'ocp_deploy.py' )

__Thank you for your time!__